##Install Libraries  - Tranformers

In [2]:
!pip install Transformers

     |████████████████████████████████| 1.8MB 15.3MB/s 
     |████████████████████████████████| 890kB 59.0MB/s 
     |████████████████████████████████| 2.9MB 51.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=e73e6b9e81a3f294f058463aad8d916be5e1eb3ee12773d44a283a0f0ea9c5ef
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


##Import libaries 


In [3]:
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    dev = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    dev = torch.device("cpu")


#Model
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
model = model.to(dev)
#Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

#Read test file from Github

import requests
url = "https://raw.githubusercontent.com/newspap/MyRepo/main/BERT_for_dummies.txt"
resp = requests.get(url)
input_ = resp.text.split("\r\n") 
input_ = list(filter(lambda x: len(x) > 5, input_))


There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [4]:
from random import sample 
input_ = resp.text.split("\r\n") 
input_ = list(filter(lambda x: len(x) > 5, input_))
print(sample(input_, 10))

['Image for post', 'Sign up for The Daily Pick', 'Conclusion', 'Image for post', 'BERT is basically a trained Transformer Encoder stack, with twelve in the Base version, and twenty-four in the Large version, compared to 6 encoder layers in the original Transformer we described in the previous article.', 'A Medium publication sharing concepts, ideas, and codes.', 'When we use the trained model to predict the intents on the unseen test dataset, the confusion matrix clearly shows how the model overfits to the majority �flight� class.', 'Follow', 'In this article, we will demonstrate Transformer, especially how its attention mechanism helps in solving the intent classification task by learning contextual relationships. After demonstrating the limitation of a LSTM-based classifier, we introduce BERT: Pre-training of Deep Bidirectional Transformers, a novel Transformer-approach, pre-trained on large corpora and open-sourced. The last part of this article presents the Python code necessary fo

##Continue ask questions

In [ ]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

def group_words(s, n):
    words = s.split()
    for i in range(0, len(words), n):
        yield ' '.join(words[i:i+n])

def get_loss(ans):
    return ans.get('loss')

while True:
    # if doc changed, reload it

    answers=[]
    question = input(bcolors.OKBLUE + "Question: " + bcolors.ENDC)

    if question == '' :
     
        break
    else:
        for longParagraph in input_:
            for paragraph in list(group_words(longParagraph, 180)):
                #encoding = tokenizer.encode_plus(text=question,text_pair=paragraph)

                #inputs = encoding['input_ids']  #Token embeddings
                #sentence_embedding = encoding['token_type_ids']  #Segment embeddings
              
                inputs = tokenizer(question, paragraph, return_tensors='pt')
                tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'].tolist()[0]) #input tokens
                start_positions = torch.tensor([1])
                end_positions = torch.tensor([3])

                #outputs = model(input_ids=torch.tensor([inputs]), token_type_ids=torch.tensor([sentence_embedding]))
                with torch.no_grad():
                    inputs = inputs.to(dev)
                    start_positions = start_positions.to(dev)
                    end_positions = end_positions.to(dev)
                    outputs = model(**inputs, start_positions=start_positions, end_positions=end_positions)
                loss = outputs.loss
                #start_scores, end_scores
                start_index = torch.argmax(outputs.start_logits)

                end_index = torch.argmax(outputs.end_logits)

                answer = ' '.join(tokens[start_index:end_index+1])

                corrected_answer = ''

                for word in answer.split():
                    
                    #If it's a subword token
                    if word[0:2] == '##':
                        corrected_answer += word[2:]
                    else:
                        corrected_answer += ' ' + word
                #if (corrected_answer.startswith(" [")):
                #    print(corrected_answer)
                lossVal = loss.item()
                
                if (len(corrected_answer) > 0):
                    print(bcolors.FAIL + str(lossVal) + ": " + bcolors.ENDC + bcolors.WARNING + corrected_answer + bcolors.ENDC) 
                    #print(paragraph)
                answers.append({"loss":lossVal, "answer":corrected_answer, "paragraph": paragraph})
    answers.sort(key=get_loss)
    print("-------------------------- Question -----------------------------------------------------")
    print(":: " + question)
    print("------------------------- Answer  ------------------------------------------------------")
    
    for ans in answers[-15:]:
        coloredAns = (bcolors.OKGREEN if ans['loss'] > 10 else bcolors.FAIL) + str(ans['loss']) + ": " + bcolors.ENDC + bcolors.WARNING + ans['answer'] + bcolors.ENDC

        print(coloredAns)
        startIdx = ans['paragraph'].lower().find(ans['answer'])

        ansLength = len(ans['answer'])

        if (startIdx != -1):
            hightlightedPara = ans['paragraph'][0:startIdx] + bcolors.WARNING + \
                    ans['paragraph'][startIdx:startIdx + ansLength] + bcolors.ENDC
            if (startIdx + ansLength < len(ans['paragraph'])):
                hightlightedPara += ans['paragraph'][startIdx + ansLength]
        else :
            hightlightedPara = ans['paragraph']
        print(hightlightedPara)

    print("############################### For Question ###############################")
    print(bcolors.HEADER +  question  + bcolors.ENDC)
    print()

Question: what is the topic?
11.030708312988281:  motivation
8.805683135986328:  classify queries into specific intents in order to generate the most coherent response . intent classification
9.975690841674805:  question answering
7.8452959060668945:  understanding the intuition
10.174060821533203:  intent labeling
9.011407852172852:  intent classification
11.935047149658203:  intent classification
8.513883590698242:  training dataset , we have 26 distinct intents , whose distribution is shown below . the dataset is highly unbalanced , with most queries labeled as flight
7.930637359619141:  image for post
9.21318531036377:  multi - class classifier
9.421148300170898:  classification
7.930637359619141:  image for post
8.305319786071777:  class flight
9.858010292053223:  confusion matrix clearly shows how the model overfits to the majority flight class
7.930637359619141:  image for post
12.315897941589355:  data augmentation
11.203878402709961:  dealing with an imbalanced dataset is a co